In [6]:
import pandas as pd
import numpy as np
import re
from transformers import BertTokenizer, BertForMaskedLM
import torch
from transformers import AdamW
from tqdm import tqdm
import gensim
from gensim.models import Word2Vec 
from gensim.models import KeyedVectors
import pandas as pd
from nltk.tokenize import RegexpTokenizer
import pickle
import ast
import statistics as s
from fuzzywuzzy import fuzz
from nltk.corpus import stopwords
import nltk
from transformers import pipeline
import copy
from nltk.corpus import stopwords

In [7]:
# load word2vec model
#w2v_model = ...
# load BERT model
#BERT_model = ...
# load dataframe
#df = ...

tokenizer = BertTokenizer.from_pretrained("GroNLP/bert-base-dutch-cased")
BERT_model = BertForMaskedLM.from_pretrained("GroNLP/bert-base-dutch-cased")
#w2v_model.intersect_word2vec_format(r"combined-160.txt", binary=False, lockf=1.0)
# https://github.com/clips/dutchembeddings

df = pd.read_csv('preprocessed_df100.csv')

In [8]:
df['gt for training'] = df['gt sentences matched']

In [9]:
# create vocabulary lists
vocab_BERT = list(tokenizer.vocab.keys())
print(len(vocab_BERT))
train = df[df['set']=='train']
#df['gt for training'] = df['gt text']
train = '.'.join(list(train['gt for training']))
train = (train.replace('.', '')).split(' ')
vocab_BERT = vocab_BERT + train

model = KeyedVectors.load_word2vec_format(r"combined-160.txt", binary=False)
vocab_word2vec = list(model.vocab.keys())
print(len(vocab_word2vec))
vocab_word2vec = vocab_word2vec + train
print(len(vocab_BERT))
print(len(vocab_word2vec))

30073
1442950
67231
1480108


In [10]:
# create list of historical expressions
with open('dictionaryBasedDutchDictionary_1.0.type_frequency.txt', 'r', encoding="utf8") as f: # part 1 historical expressions
    words = f.readlines()
hist_words1 = [word[0 : word.index("\t")] for word in words]
#print(woorden)

with open('dutchCorpusBasedDictionary_1.1.tf.txt', 'r', encoding="utf8") as f: # part 2 historical expressions
    words = f.readlines()
hist_words2 = [word[0 : word.index("\t")] for word in words]
#print(woorden)

expressions = list(set(hist_words1).union(set(hist_words2))) # all historical expressions

with open('wordlist.txt', 'r', encoding="utf8") as f: # modern_vocab
    words = f.read().split('\n')
    #words3 = [word[0 : word.index("\n")] for word in words]
    modern_vocab = words
    
# historical expressions is everything in hist_expression that is not in modern_vocab
hist_expressions = list(set(expressions).difference(set(modern_vocab)))
print(hist_expressions[:100])
print(len(hist_expressions))
print(len(modern_vocab))

['Bloedspuwingen', 'tón-bèèzen', 'menschengemeenschap', 'ieiunium', 'jong-getal', 'Nassouwe', 'polslag', 'mutamur', "rey'", 'vogelpruimbast', 'Tractrat', 'synaesthetische', 'sausbaars', 'kievitseijeren', 'imbijte', 'impossibele', 'gerepouseert', 'chocolaadje', 'vlierkappers', 'zielsverheffende', 'uitstrominge', 'reuzentoppen', 'Arminiaanschen', 'natuureigenste', 'vlotafstand', 'aptae', 'latynsche', 'anaemische', 'avanturen', 'lauas', 'toovernij', 'beweluen', 'Geschreven', 'Gon', 'wijdloopige', 'racaille', 'duynbosjes', 'kacheldroogen', 'vischkotter', 'hatelike', 'Tuigt', 'voorpeynst', 'vloeiijzerwerk', 'huusmul', 'bouteinden', 'knipvoeg', 'geitekaes', 'verwoesteden', 'rosenknopje', 'vier-boete', 'ducatontjes', 'végétale', 'wippelpaard', 'sonduloet', 'lozeleur', 'volmachtzbryeve', 'omwingerdt', 'wapenscouwinge', 'U.R.S.S', 'bloss', 'weynichken', 'perceptieregel', 'dibberij', 'wegwaaijing', 'Braunbuch', 'midlantse', 'letterdienders', "gehemelt'", 'roedraegher', 'kaasverw', 'geltgierig', 

In [11]:
# create list with homonyms
with open('homoniemen.txt', 'r', encoding="utf8") as f: # part 1 historical expressions
    homonyms = f.readlines()
homonyms = [word[0 : word.index(":")] for word in homonyms]
print(homonyms[0:10])

['horen', 'vuil', 'vizier', 'militant', 'naturel', 'primetime', 'koperdraad', 'tel', 'zondagmiddag', 'plastiek']


In [12]:
# create list with infrequent expressions
# = domain expressions etc. Not in original vocab, but a few times in the new training set.
model_vocabs = list(tokenizer.vocab.keys()) + list(model.vocab.keys())
# find infrequent expressions
# merge all GT text to list of words
# count unique words
gt_texts = df['gt for training'].to_list()
gt_texts = '.'.join(gt_texts)

import re
#regex = re.compile('[^a-zA-Z]')
#gt_texts = regex.sub('', gt_texts)
gt_texts = re.sub(r'[^A-Za-z ]+', '', gt_texts).lower()
gt_texts = gt_texts.split(' ')

#print(gt_texts)
from collections import Counter
occurrences = Counter(word for word in gt_texts)
occurrences_counts = Counter(occ for occ in occurrences.values())
infrequent_expressions = [key for key , value in occurrences.items() if value == 1]

In [14]:
def process(tokens_list):
    tokens_list = [re.sub(r'[^\w\d\s\-]+', '', x) for x in tokens_list]
    tokens_list = [x.lower() for x in tokens_list]
    return tokens_list

# make sure all lists are in lowercase and without punctuation except for hyphens
# homonyms
homonyms = process(homonyms)
# BERTs vocab:
vocab_BERT = process(vocab_BERT)
# word2vecs vocab:
vocab_word2vec = process(vocab_word2vec)
# historical expressions
hist_expressions = process(hist_expressions)
# the modern vocabulary
modern_vocab = process(modern_vocab)
# infrequent expressions
infrequent_expressions = process(infrequent_expressions)
# the dictionary for the baseline model
dictionary = process(modern_vocab + hist_expressions)

In [15]:
# save al lists and later unpack:
all_lists_tokens = [homonyms, vocab_BERT, vocab_word2vec, hist_expressions, modern_vocab, infrequent_expressions, dictionary]

with open('all_lists_tokens.txt', 'wb') as f:
    pickle.dump(all_lists_tokens, f)